<a href="https://colab.research.google.com/github/rajimitara/twitter-sentiment/blob/main/twitter_sentitment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Text classifier**

In [ ]:
import nltk
nltk.download('all')

In [7]:
import numpy as np
import pandas as pd
import re

In [74]:
dataset = pd.read_csv('https://raw.githubusercontent.com/rajimitara/twitter-sentiment/main/Corona_NLP_train.csv')

In [135]:
#Get kaggle data for twitter sentiment on vaccine
dataset.head(20)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [10]:
ps = PorterStemmer()

In [392]:
corpus = [] 

In [393]:
for i in range(len(dataset)):
  tweet = dataset['OriginalTweet'][i]
  tweet = tweet.lower()
  tweet = re.sub(r'\s+',' ',tweet)
  tweet = re.sub(r'^[a-zA-Z]',' ',tweet)
  corpus.append(tweet)

In [ ]:
corpus

In [395]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 150, min_df = 3, max_df = 0.6)

In [396]:
X = tfidf.fit_transform(corpus).toarray()

In [ ]:
y = dataset.Sentiment.factorize()[0]
y

In [398]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [399]:
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors=5)
classifierKNN.fit(X_train,y_train)

KNeighborsClassifier()

In [400]:
y_pred_knn = classifierKNN.predict(X_test)

In [401]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_knn
print("KNN Accuracy score", accuracy_score(y_test, y_pred_knn))

KNN Accuracy score 0.4444444444444444


In [403]:
sample1 = ['Vaccine is best best best best precaution measure']
sample2 = ['Vaccine is not helpful always panic']

In [404]:
sample1 = tfidf.transform(sample1).toarray()
sample2 = tfidf.transform(sample2).toarray()

In [407]:
sentiment1 = classifierKNN.predict(sample1)
sentiment1


array([0])

In [408]:
sentiment2 = classifierKNN.predict(sample2)
sentiment2

array([2])

**Twitter sentiment live prediction**

In [409]:
import tweepy

In [410]:
consumer_key='LEyiXvdcqVd1hOQacbBhvP0Mk'
consumer_secret='qqxTIhBfbowDg4xHSsaguJCt4orfhrhfIS6ZKYPBreDAseOrl1'
access_token='1515671963376791552-ulGIWc5zJVV2H8tuI13lGfpH2L3Yat'
access_secret='Ac2j4UnXtOljvHNcN5nCcICmg9tncV9Celpc3KqB0wVwW'

In [411]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [412]:
api = tweepy.API(auth, timeout=20)

In [413]:
tweet_text = 'vaccine'
tweets_list = []

In [ ]:
for status in tweepy.Cursor(api.search, q = tweet_text, lang = 'en', result_type = 'recent').items(500):
      tweets_list.append(status.text)

In [71]:
len(tweets_list)

0

In [72]:
for i in range(len(tweets_list)):
  tweet = re.sub(r'\W',' ',tweets_list[i])
  tweet = re.sub(r'\s+',' ',tweets_list[i])
  tweet = re.sub('[^a-zA-Z]',' ',tweets_list[i])
  tweet = tweet.lower()
  tweet = tweet.split()
  clean_tweet = [ps.stem(word) for word in customer_review if not word in set(stopwords.words('english'))]
  clean_tweet = ' '.join(clean_tweets)
  tweets_list[i] = clean_tweet

In [ ]:
tweets_list[9]

In [ ]:
#load tf-idf model and text classifier


In [ ]:
#predict live tweets
for tweet in twist_list:
    sentiment = classifierKNN.predict(tfidf.transform([tweet]).toarray())
    if sentiment[0] == 1:
      positive_tweet +=1
    else:
      negative_tweet +=1   